In [1]:
# 네이버에서 검색어 입력받아 검색 한 후 블로그 메뉴를 선택하고
# 오른쪽에 있는 검색옵션 버튼을 눌러서
# 정렬 방식과 기간을 입력하기

#Step 0. 필요한 모듈과 라이브러리를 로딩합니다.
import sys # 시스템
import os  # 시스템

import pandas as pd  # 판다스 : 데이터분석 라이브러리
import numpy as np   # 넘파이 : 숫자, 행렬 데이터 라이브러리

from bs4 import BeautifulSoup    # html 데이터를 전처리
from selenium import webdriver   # 웹 브라우저 자동화
import time    # 서버와 통신할 때 중간중간 시간 지연. 보통은 1초
from tqdm import tqdm_notebook   # for문 돌릴 때 진행상황을 %게이지로 알려준다.

In [2]:
driver = webdriver.Chrome("chromedriver.exe")

In [3]:
# 네이버 영화 사이트로 이동
driver.get('https://movie.naver.com/')

In [4]:
# 네이버 영화에서 검색하기
query = '그 시절, 우리가 좋아했던 소녀'

searchElm = driver.find_element_by_id("ipt_tx_srch")
searchElm.click()
searchElm.send_keys(query)
searchBtnElm = driver.find_element_by_css_selector(".btn_srch")
searchBtnElm.click()

In [5]:
# 검색결과 화면에서 url 찾아서 이동하기
itemElms = driver.find_elements_by_css_selector(".search_list_1 > li > dl > dt > a")
url = None
for itemElm in itemElms:
    title = itemElm.text
    if title.find(query) > -1:
        url = itemElm.get_attribute("href")
driver.get(url)

In [6]:
# 평점 화면으로 이동해서 iframe 들어가기
scoreElm = driver.find_element_by_css_selector("#movieEndTabMenu > li:nth-child(5) > a")
scoreUrl = scoreElm.get_attribute("href")
driver.get(scoreUrl)
driver.switch_to.frame("pointAfterListIframe")

In [7]:
def preprocess(scoreElm):
    dict = {}
    dict["score"] = scoreElm.find_element_by_css_selector(".star_score > em").text
    dict["text"] = scoreElm.find_element_by_css_selector(".score_reple > p > span").text
    return dict

In [8]:
# 평점 데이터 저장하기 # 페이지 넘어가면서
reviews = []
halted = False
maxiumPage = 10
presPageCount = 0
while halted == False:
    
    # 최대 페이지 제한 # 페이지가 너무 많아서 제한기능을 추가함
    presPageCount += 1
    if presPageCount > maxiumPage:
        break;
        
    # 현재 페이지의 리뷰를 가져와서 저장
    scoreElms = driver.find_elements_by_css_selector(".score_result > ul > li")
    for scoreElm in scoreElms:
        reviews.append(preprocess(scoreElm))
    
    # 페이지 영역에서 다음 페이지 결정하기
    presPageElm = None
    nextPageElm = None
    pageElms = driver.find_elements_by_css_selector(".paging > div > a")
    for pageElm in pageElms:
        if presPageElm != None:
            nextPageElm = pageElm;
            break;
        onElms = pageElm.find_elements_by_css_selector("span")
        if len(onElms) == 0:
            continue;
        ison = onElms[0].get_attribute("class");
        if ison == "on":
            presPageElm = pageElm
    
    # 다음 페이지 이동하기 # 다음 페이지가 없으면 종료
    if nextPageElm == None:
        halted = True
    else:
        nextPageElm.click()

In [9]:
# 수집된 리뷰들 전처리하여 저장하기
print(len(reviews))
reviews

100


[{'score': '10',
  'text': '건축학개론과 비교하지 말아주삼둘 다 첫사랑영화인데 건축학개론은 공감도안되고 설레지도 않았음이건 정말 설레는 영화임'},
 {'score': '10', 'text': '보고나서 정말로 마음이 아련해지는 영화...'},
 {'score': '10', 'text': '어쩌면 이루워지지 않아서 더 아름다운영화'},
 {'score': '10', 'text': '나도 학창시절에 저런 여학생있었으면 공부 열심히 했을텐데'},
 {'score': '10',
  'text': '첫사랑이 아직도 생각나는 사람이라면 꼭 봐야할 영화 !!건축학개론이랑 비교불가인 그 영화!!'},
 {'score': '10',
  'text': '많은 하이틴 영화에서 갑이 아닐까 싶다나도 저 로맨스에 주인공이였으면 하는 생각!!!!!!..........'},
 {'score': '10', 'text': '전세계에 존재하는 첫사랑 영화중에 최고일듯'},
 {'score': '10', 'text': '사랑은 타이밍'},
 {'score': '10',
  'text': '"사람들이 그러지사랑은 알듯 말듯 한 순간이 아름답다고진짜 둘이 하나가 되면 많은 느낌이 사라지고 없대그래서 오래도록 날 좋아하게 두고 싶었어."..'},
 {'score': '10', 'text': '새벽에 혼자 보지말것. 잠도 못자고 감수성의 홍수에서 헤어날 수 없다'},
 {'score': '10', 'text': '성장하는동안 가장 잔인한건 여학생은 남학생보다 성숙하며 그 성숙함에 견뎌낼 남학생은 없다'},
 {'score': '10',
  'text': '신랑하게 대리만족하는 키스신이아녔어~~~결혼 할 수도 있었다는 행복한 상상을 잠시 하는거였다..그 때 찾아가 사과만 했더라면..우는장면과 사과하는 장면이 오버랩되는 씬이 제일 슬프네요ㅠㅜ정말짱~~'},
 {'score': '10', 'text': '기자랑 평론가는 돈을 안받으면 저따구로 평가하는구나... 영화 보긴 봤니?'},